In [1]:
import pandas as pd
import numpy as np 
import pickle 

corrMatrix = pickle.load(open('corrmtx.pkl','rb'))
model = pickle.load(open('model.pkl','rb'))
df = pd.read_csv('Users.csv')

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
customer_data = pd.read_csv('ColumnCountsperCustomer.csv')

In [4]:
def get_similar(itemId):
    similar_ratings = corrMatrix[itemId]
    index_list=similar_ratings.index
    value_list = similar_ratings.values
    final_df = pd.DataFrame(index=index_list)
    final_df['Corr'] = value_list
    final_df = final_df.sort_values(by='Corr',ascending=False)
    return final_df

In [5]:
def get_item_item(test_user) :
    user_items = []
    data = customer_data[customer_data['CustomerNumber']==test_user]
    user_items = [column for column in data.columns if data[column].iloc[0]!=0][1:]
    similar_items = pd.DataFrame()
    for item in user_items:
        recommended_df = get_similar(item)
#         print(recommended_df)
        items = recommended_df.index
        for item in items:
            if item not in user_items:
                similar_items = similar_items.append(recommended_df.loc[item])
    similar_items = similar_items[similar_items['Corr']>0]
    final_item_list = []
    final_similarity = []

    for item in np.unique(similar_items.index):
        data = similar_items.loc[item]
        final_item_list.append(item)
        final_similarity.append(data.Corr.mean())
    
    items_df = pd.DataFrame()
    items_df['Item'] = final_item_list
    items_df['Similarity'] = final_similarity
    x = np.asanyarray(final_similarity,dtype=np.float32)
    x-=np.min(final_similarity)
    x/= (np.max(final_similarity)-np.min(final_similarity))
    items_df['Score'] = x
    items_df = items_df.sort_values(by='Score',ascending=False)
    return items_df
        

In [7]:
def get_item_users(test_user):
    distances,indices = model.kneighbors(df[df['Customer Code']==test_user][['DaysPresent','Revenue']],n_neighbors=6)
    indices = indices.squeeze()[1:]
    distances = distances.squeeze()[1:]
    similar_users=[df['Customer Code'].values[i] for i in indices]
    user_items = []
    data = customer_data[customer_data['CustomerNumber']==test_user]
    user_items = [column for column in data.columns if data[column].iloc[0]!=0]
    recommended_items= []
    similarity = []
    for i in range(len(similar_users)) :
        if similar_users[i] in list(customer_data['CustomerNumber']):
            data = customer_data[customer_data['CustomerNumber']==similar_users[i]]
#         print(data.head())
            for column in data.columns[1:] :
                if data[column].iloc[0]!=0:
                    if column not in user_items:
                        recommended_items.append(column)
                        similarity.append(distances[i])
                        
    users_df = pd.DataFrame()
    users_df['Item'] = recommended_items
    users_df['Similarity'] = similarity
    
    x = np.asanyarray(similarity,dtype=np.float32)
    x-=np.min(similarity)
    x/= (np.max(similarity)-np.min(similarity))
    x = 1-x
    users_df['Score'] = x
    users_df = users_df.sort_values(by='Score',ascending=False)    
    return users_df

In [8]:
def get_recommendation(userID) :
    item_item_recommendation = get_item_item(userID)
    user_item_recommendation = get_item_users(userID)
    final_item_list =[]
    score = []
    confidence = []
    
    for item in item_item_recommendation.Item.values:
        if item in user_item_recommendation.Item.values:
            final_item_list.append(item)
            confidence.append(1)
            if user_item_recommendation[user_item_recommendation['Item']==item].Score.values[0] > item_item_recommendation[item_item_recommendation['Item']==item].Score.values[0] :           
                score.append(user_item_recommendation[user_item_recommendation['Item']==item].Score.values[0])
            else:
                score.append(item_item_recommendation[item_item_recommendation['Item']==item].Score.values[0])
        else :
            final_item_list.append(item)
            confidence.append(0)
            score.append(item_item_recommendation[item_item_recommendation['Item']==item].Score.values[0])
    recommendation_df = pd.DataFrame()
    recommendation_df['Items'] = final_item_list
    recommendation_df['Confidence'] = confidence 
    recommendation_df['Score'] = score
    recommendation_df = recommendation_df[recommendation_df['Score']>0]
    recommendation_df=recommendation_df.sort_values(['Confidence', 'Score'], ascending=[False, False])
    
#     print(recommendation_df.head())
    return recommendation_df

In [9]:
recommended = get_recommendation('CAA0023')

In [10]:
recommended

,Items,Confidence,Score
7,DP02,1,1.000000
97,AS17,1,1.000000
163,CA06,1,0.464520
168,MN01,1,0.464520
107,GH02,1,0.258284
...,...,...,...
215,RW09,0,0.060731
216,RW01,0,0.057175
217,RW10,0,0.040671
218,AS30,0,0.024575
